In [1]:
import transformers 
import bert_score

/Users/au672746/Library/CloudStorage/OneDrive-Aarhusuniversitet/Skrivebord/monopolet-nlp/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from bert_score import score

In [33]:
advice_one_congr = 'Det bedste vil være at tage en ærlig snak med din leder om, hvor meget du har på tallerkenen lige nu. Forklar, at du gerne vil påtage dig projektet, men at du kun kan gøre det, hvis der bliver prioriteret eller omfordelt nogle af dine nuværende opgaver. På den måde kan I sammen sikre, at kvaliteten af dit arbejde ikke lider.'
advice_two_congr = 'Måske kunne det være en god idé først at drøfte din nuværende belastning med din leder. Ved at dele dine bekymringer og behov skaber du mulighed for fælles forventningsafstemning og en løsning, der både tager højde for projektets betydning og din trivsel.'

In [7]:
P, R, F1 = score(advice_one_congr, advice_two_congr, lang='da', verbose=True)

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  2.75it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 113.90it/s]

done in 0.38 seconds, 2.67 sentences/sec


In [10]:
F1

tensor([0.6864])

In [19]:
advice_one_incongr = 'Det kan være hjælpsomt først at tage udgangspunkt i, hvad du realistisk kan rumme lige nu. Hvis projektet presser dig ud over din grænse, kan den mest ansvarlige beslutning være at takke nej, så du bevarer overskud og kvalitet i de opgaver, du allerede håndterer.'
advice_two_incongr = 'Du kan også vælge at fokusere på, hvordan projektet kan blive håndterligt, hvis rammerne justeres. I den situation kan du sige ja, men kun hvis du får ekstra støtte, mere tid eller en tydelig prioritering af dine eksisterende opgaver.'

In [17]:
P_in, R_in, F1_in = score(advice_one_incongr, advice_two_incongr, lang='da', verbose=True)

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 558.64it/s]

done in 0.51 seconds, 1.94 sentences/sec


In [18]:
F1_in

tensor([0.7372])

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = [advice_one_incongr, advice_two_incongr]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[ 0.0088,  0.1429, -0.0135,  ...,  0.0684, -0.0101, -0.0032],
        [ 0.0272,  0.1489, -0.0135,  ...,  0.0822, -0.0594, -0.0096]])


In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
sentence_embeddings[0].shape

torch.Size([768])

In [30]:
import numpy as np

In [31]:
cosine_similarity(np.array(sentence_embeddings[0]).reshape(1, -1), np.array(sentence_embeddings[1]).reshape(1, -1))

/var/folders/m8/6qm1yrz575b68s5y27_5z0jc33spwq/T/ipykernel_65311/3400734062.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  cosine_similarity(np.array(sentence_embeddings[0]).reshape(1, -1), np.array(sentence_embeddings[1]).reshape(1, -1))


array([[0.76294506]], dtype=float32)

In [34]:
sentences = [advice_one_congr, advice_two_congr]

# Load model from HuggingFace Hub
#tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
#model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[-0.0283,  0.1609, -0.0128,  ...,  0.0806, -0.0390, -0.0081],
        [-0.0291,  0.1546, -0.0121,  ...,  0.0348, -0.0512,  0.0305]])


In [35]:
cosine_similarity(np.array(sentence_embeddings[0]).reshape(1, -1), np.array(sentence_embeddings[1]).reshape(1, -1))

/var/folders/m8/6qm1yrz575b68s5y27_5z0jc33spwq/T/ipykernel_65311/3400734062.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  cosine_similarity(np.array(sentence_embeddings[0]).reshape(1, -1), np.array(sentence_embeddings[1]).reshape(1, -1))


array([[0.8297918]], dtype=float32)